In [3]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# desired single-level columns (yfinance standard)
desired_cols = ['symbol','Close', 'High', 'Low', 'Open', 'Volume']
df_all = pd.DataFrame(columns=desired_cols)
tickers = [
    "ADANIENT",
    "ADANIPORTS",
    "APOLLOHOSP","ASIANPAINT","AXISBANK","BAJAJ-AUTO","BAJFINANCE",
    "BAJAJFINSV","BEL","BHARTIARTL","CIPLA","COALINDIA","DRREDDY","EICHERMOT","ETERNAL",
    "GRASIM","HCLTECH","HDFCBANK","HDFCLIFE","HINDUNILVR","HINDALCO","ICICIBANK","INFY",
    "INDIGO","ITC","JIOFIN","JSWSTEEL","KOTAKBANK","LT","M&M","MARUTI","MAXHEALTH",
    "NESTLEIND","NTPC","ONGC","POWERGRID","RELIANCE","SBIN","SBILIFE","SHRIRAMFIN",
    "SUNPHARMA","TATACONSUM","TATAMOTORS","TATASTEEL","TCS","TECHM","TITAN","TRENT",
    "ULTRACEMCO","WIPRO"
]
start_date = "2025-04-25"
end_date = "2025-10-25"
nse_tickers = [s.strip().upper() + ".NS" if not (s.endswith(".NS") or s.endswith(".BO")) else s for s in tickers]
for symbol in nse_tickers:
    # data = yf.download(symbol, period="1mo")
    data = yf.download(symbol, start=start_date, end=end_date)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    data['symbol'] = symbol.replace('.NS','').replace('.BO','')
    df_all = pd.concat([df_all, data], axis=0, ignore_index=False)
df_all = df_all.sort_index().reset_index()
df_all = df_all.sort_values(['symbol', 'index'], ascending=[True, True]).reset_index(drop=True)
df_all['Day_Change'] = df_all.groupby('symbol')['Close'].diff()
df_all['Day_Change_%'] = df_all.groupby('symbol')['Close'].pct_change().mul(100).round(2)
df_all = df_all.sort_values(['symbol', 'index'], ascending=[True, False]).reset_index(drop=True)
df_all['Date']=df_all['index']
df_all['Date'] = pd.to_datetime(df_all['Date'])

# resample weekly per symbol, taking last close of each week
weekly = (
    df_all.set_index('Date')
    .groupby('symbol')['Close']
    .resample('W-FRI')
    .last()
    .reset_index()
)

# compute weekly percentage change
weekly['Weekly_Return_%'] = weekly.groupby('symbol')['Close'].pct_change().mul(100).round(2)
weekly = weekly.sort_values(['symbol', 'Date'], ascending=[True, False]).reset_index(drop=True)
# Ensure date column is datetime
df_all['Date'] = pd.to_datetime(df_all['Date'])

# Resample monthly per symbol (take last close of each month)
monthly = (
    df_all.set_index('Date')
    .groupby('symbol')['Close']
    .resample('M')     # 'M' = month-end frequency
    .last()
    .reset_index()
)

# Compute month-over-month percentage change
monthly['Monthly_Return_%'] = (
    monthly.groupby('symbol')['Close']
    .pct_change()
    .mul(100)
    .round(2)
)

# Optional: Add formatted string column for display (like +5.62%)
monthly['Monthly_Return_Str'] = monthly['Monthly_Return_%'].apply(
    lambda x: f"{x:+.2f}%" if pd.notna(x) else ""
)

monthly = monthly.sort_values(['symbol', 'Date'], ascending=[True, False]).reset_index(drop=True)
summary_df1 = (
    df_all.groupby('symbol')['Day_Change_%']
    .apply(lambda x: list(x.head(10)))
    .reset_index(name='5D_Change')
)
# summary_df1

summary_df2 = (
    weekly.groupby('symbol')['Weekly_Return_%']
    .apply(lambda x: list(x.head(5)))
    .reset_index(name='5W_Change')
)
summary_df3 = (
    monthly.groupby('symbol')['Monthly_Return_%']
    .apply(lambda x: list(x.head(5)))
    .reset_index(name='5M_Change')
)
summary_df = summary_df1.merge(summary_df2, on='symbol').merge(summary_df3, on='symbol')
import os

# Create directory if it doesn't exist
save_dir = r'D:\dashboard\dashboard_project_full_data\dashboard_project_python_only\all_data'
os.makedirs(save_dir, exist_ok=True)
# Save the file
summary_df.to_csv(os.path.join(save_dir, 'day_week_month_summery.csv'), index=False)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********